In [ ]:
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Indexing

## 0. Preparación

### Embedding model

In [2]:
import getpass
import os

#### Using OpenAI

In [3]:

os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY_RAG")

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### Using Google Vertex

In [4]:
# Set your GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rodrigo/DataSci/FKTech/credentials/fk-rag-3564a80296be.json"

In [5]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-004")

### Vector store

In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

## 1. Load

In [7]:
# Load contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

## 2. Split

In [8]:
# Use text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

Veamos cuántos splits hizo

In [ ]:
len(all_splits)

Salvemos estos documentos en la base vectorial

In [10]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)


### Exploremos la base construida

Veamos cuántos elementos tiene esta base

In [ ]:
print(vector_store._collection.count())

Preparemos una pregunta e busquemos los chunks más parecidos

In [12]:
pregunta = 'Please explain Chain of Thought'

In [13]:
chunks = vector_store.similarity_search(pregunta, k=3 # Definimos la cantidad de fragmentos a recuperar
           #                         filter={"source":"documento_especifico.pdf"} # Filtro para restingir en base a metadatos la recuperación
)


1. **`vector_store.similarity_search`**:  
   - Este método realiza una búsqueda basada en similitudes en la base de datos vectorial. Compara la consulta (representada como un vector) con los vectores almacenados en la base de datos para encontrar los documentos más similares.

2. **`question`**:  
   - Representa la consulta del usuario. Es el texto o el vector con el que se buscan coincidencias en la base de datos. Puede ser una pregunta o cualquier cadena de texto relevante.

3. **`k=5`**:  
   - Especifica el número de documentos más similares que se desean recuperar. En este caso, se recuperarán los 5 documentos con mayor similitud. Ajustar este parámetro puede cambiar la cantidad de resultados devueltos.

4. **`filter`** (comentado en el código):  
   - Es un filtro opcional para limitar los documentos recuperados según criterios específicos. Por ejemplo:
     - `filter={"source": "documento_especifico.pdf"}` restringiría los resultados solo a aquellos documentos cuya fuente coincida con `"documento_especifico.pdf"`.
   - Los filtros son útiles para buscar similitudes dentro de un subconjunto de documentos.

In [ ]:
import textwrap
textwrap.wrap(chunks[0].page_content)

## 3. Generate

In [15]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-flash")

Usamos un prompt genérico sacado del Hub de Langchain, pero pensado para RAGs

In [16]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
question = "¿Me podrías explicar qué es Chain of Thought y darme la cita del paper que lo presenta por primera vez?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)

In [ ]:
textwrap.wrap(answer.content)

---

Si queremos hacer una aplicación, podemos definir los estados de la misma manera.

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}